In [ ]:
# app.py (Flask后端)
from flask import Flask, render_template, jsonify
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from datetime import timedelta

app = Flask(_name_)

def get_predictions():
    # 数据加载和预处理
    df = pd.read_csv("kuala-lumpur-air-quality.csv")
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')
    df.columns = df.columns.str.strip()

    # 生成滞后特征
    def create_lag_features(data, lag=30):
        for i in range(1, lag+1):
            data[f'lag_{i}'] = data['aqi'].shift(i)
        return data

    df = create_lag_features(df,90)#use last 90 days data
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df.dropna()

    # 准备训练数据
    X = df.drop(columns=['date', 'aqi'])
    y = df['aqi']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # 训练模型
    model = RandomForestRegressor(
        n_estimators=200,
        criterion='squared_error',
        max_depth=20,
        max_features='sqrt',
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # 生成预测
    future_dates = pd.date_range(df['date'].max() + timedelta(days=1), periods=7)
    future_data = pd.DataFrame({'date': future_dates})
    last_known_data = df.iloc[-90:][['aqi']].values.flatten().tolist()

    for i in range(7):
        feature_row = last_known_data[-90:]
        X_pred = pd.DataFrame([feature_row], columns=X_train.columns)
        pred = model.predict(X_pred)[0]
        last_known_data.append(pred)
        future_data.loc[i, 'aqi_pred'] = pred

    return future_data

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/api/predict')
def predict():
    try:
        future_data = get_predictions()
        predictions = [{
            'date': row['date'].strftime('%Y-%m-%d'),
            'aqi_pred': round(row['aqi_pred'], 2)
        } for _, row in future_data.iterrows()]
        return jsonify(predictions)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if _name_ == '_main_':
    app.run(debug=True)